# A Jupyter kernel for bash

- [github repo](https://github.com/takluyver/bash_kernel)

```sh
cat dog.png | display
echo "<b>Dog</b>, not a cat." | displayHTML
echo "alert('It is known khaleesi\!');" | displayJS
```

In [3]:
ls

README.md		python-workspace.ipynb	run-docker-notebook-d.sh
bash-workspace.ipynb	run-docker-bash.sh	run-docker.sh
golang-workspace.ipynb	run-docker-d.sh
